# Set up and load data:

In [25]:
%load_ext autoreload
%autoreload 2
import numpy as np
from tqdm import tqdm
from functions.seq_autoencoder import *
setup_gpus()
ds_all, ds_all_centered, datasets, datasets_centered, ds_counts = load_data()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
loading: betternot_and_retrograde
	 Shape: (9925, 53, 3)
	 Min: [-1.96127391 -4.00212193 -0.15710293]
	 Max: [2.1591475  3.52660036 2.09531498]
loading: beyond
	 Shape: (5803, 53, 3)
	 Min: [-2.18821788 -2.68805718 -0.06788409]
	 Max: [2.1671412  3.16986203 2.16449094]
loading: chunli
	 Shape: (3866, 53, 3)
	 Min: [-1.50739741 -2.62433457 -0.08408955]
	 Max: [1.38144195 1.92739224 2.07176852]
loading: honey
	 Shape: (5309, 53, 3)
	 Min: [-2.40029621 -4.7584796  -0.12750754]
	 Max: [0.95588988 3.2632885  2.06287146]
loading: knownbetter
	 Shape: (6649, 53, 3)
	 Min: [-2.42928696 -3.50348401 -0.48134506]
	 Max: [2.13536358 3.36868167 2.0231936 ]
loading: penelope
	 Shape: (6757, 53, 3)
	 Min: [-2.37398148 -3.10730028 -0.15576062]
	 Max: [1.6543231  3.20413566 2.22775602]
Full data shape: (38309, 53, 3)


# Animate the real data:

Particularly cool starting frame numbers to look at:
- 11910 # tourjete + pirouettes
- 9259 # front kicks
- 21755 # penche
- 37067 # slow gestures
- 13443 # back leg extension
- 28840 # 180 hop

In [27]:
seq_len = 128
index_start = 9259
# index_start = np.random.randint(0,len(ds_all_centered)-seq_len)
print("Seeding with frame {}".format(index_start))
xtest = ds_all[index_start:index_start+seq_len]
animation = animate_stick(xtest, 
                          figsize=(10,8), 
                          cmap='inferno', 
                          cloud=False,
#                           cloud_alpha=0.035,
                          birds_eye=False
                         )
HTML(animation.to_html5_video())
# HTML(animation.to_jshtml())

Seeding with frame 9259


# Load a trained model and generate new movements:

In [11]:
seq_len      = 128
latent_dim   = 256
n_layers     = 3 
n_units      = 384 
use_dense    = True
kl_weight    = 1 
resolution   = 3e-1 
lr           = 3e-4
do_rotations = True
extrap_len   = seq_len//2

encoder, decoder, auto, _ = mk_seq_ae(ds_all, seq_len=seq_len, latent_dim=latent_dim,
                                   n_units=n_units, n_layers=n_layers,
                                  use_dense=use_dense, kl_weight=kl_weight,
                                  resolution=resolution, do_rotations=do_rotations, extrap_len=extrap_len)
K.set_value(auto.optimizer.lr, lr)

In [12]:
encoder.load_weights('weights/checkpoint_weights_vae_lstm_continued2_lr_0.001_encoder.h5')
decoder.load_weights('weights/checkpoint_weights_vae_lstm_continued2_lr_0.001_decoder.h5')
auto.load_weights('weights/checkpoint_weights_vae_lstm_continued2_lr_0.001_autoencoder.h5')
auto.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 128, 53, 3)        0         
_________________________________________________________________
rotation_layer_1 (RotationLa (None, 128, 53, 3)        0         
_________________________________________________________________
model_1 (Model)              [(None, 256), (None, 256) 3395072   
_________________________________________________________________
model_2 (Model)              (None, 128, 53, 3)        2807040   
_________________________________________________________________
rotation_layer_2 (RotationLa (None, 128, 53, 3)        0         
Total params: 6,202,112
Trainable params: 6,202,112
Non-trainable params: 0
_________________________________________________________________


# Make a duet!

In [13]:
! mkdir duet_test

In [17]:
index_start = 29441-seq_len
sigma = 0
total_vids = 25
sigmas = [0,0,0,0,1,1,0,0.25,0.25,0.5,0.5,0.75,0.75,1,1,2,2,3,4,0.75,0.5,0.25,0,0,0]
# sigmas = np.concatenate((np.linspace(0,2,12),np.linspace(2,0,12)))

for i in range(total_vids):
    print(i)
    index_start = index_start+seq_len
    xtest = ds_all_centered[index_start:index_start+seq_len]
    _, ztest, _ = encoder.predict(np.expand_dims(xtest,axis=0))
    xproj = decoder.predict(ztest + np.random.normal(0,sigmas[i],latent_dim))[0]
    animation = animate_stick(xtest,ghost=xproj, dot_alpha=0.7, ghost_shift=0.2, figsize=(12,8), cmap='inferno')
    animation.save('duet_custom/duet_'+str(i)+'_sigma'+str(sigmas[i])[:4]+'.mov', codec="png", dpi=100, bitrate=-1)

0


Skeleton lines: 100%|██████████| 74/74 [00:00<00:00, 334.49it/s]


1


Skeleton lines: 100%|██████████| 74/74 [00:00<00:00, 1302.93it/s]


2


...

Skeleton lines: 100%|██████████| 74/74 [00:00<00:00, 217.00it/s]


23


Skeleton lines: 100%|██████████| 74/74 [00:00<00:00, 1335.33it/s]


24


Skeleton lines: 100%|██████████| 74/74 [00:00<00:00, 1329.92it/s]


# Only focus on a specific video for cherry-picking

In [79]:
seq_len=128

In [80]:
index_start = np.random.randint(0,len(ds_all_centered)-seq_len)
# index_start = 29441
i = 0

In [83]:
sigma = 1
index = index_start+i*seq_len
xtest = ds_all_centered[index:index+seq_len]
_, ztest, _ = encoder.predict(np.expand_dims(xtest,axis=0))
xproj = decoder.predict(ztest + np.random.normal(0,sigma,latent_dim))[0]
animation0 = animate_stick(xtest,ghost=xproj, 
                           dot_alpha=0.7, 
                           ghost_shift=0.2, 
                           figsize=None, cmap='inferno')
HTML(animation0.to_html5_video())

TypeError: animate_stick() got an unexpected keyword argument 'dpi'

In [ ]:
sigma = 1.5
index = index_start+i*seq_len
xtest = ds_all_centered[index:index+seq_len]
_, ztest, _ = encoder.predict(np.expand_dims(xtest,axis=0))
xproj = decoder.predict(ztest + np.random.normal(0,sigma,latent_dim))[0]
animation1 = animate_stick(xtest,ghost=xproj, dot_alpha=0.7, ghost_shift=0.2, figsize=(12,8), cmap='inferno')
HTML(animation1.to_html5_video())

In [74]:
sigma = 20
index = index_start+i*seq_len
xtest = ds_all_centered[index:index+seq_len]
_, ztest, _ = encoder.predict(np.expand_dims(xtest,axis=0))
xproj = decoder.predict(ztest + np.random.normal(0,sigma,latent_dim))[0]
animation2 = animate_stick(xtest,ghost=xproj, dot_alpha=0.7, ghost_shift=0.2, figsize=(12,8), cmap='inferno')
HTML(animation2.to_html5_video())

Pick the best one, and save it...

In [75]:
print(index)

33450


In [159]:
animation1.save('duet_moreinteresting/duet_'+str(i)+'.mov', codec="png", dpi=100, bitrate=-1)